In [1]:
print("OK")

OK


In [39]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from pinecone import Pinecone, ServerlessSpec
from langchain.llms import CTransformers


In [35]:
#extract data from the pdf
def load_pdf(data):
    loader=DirectoryLoader(data,glob="*.pdf",loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [38]:
exracted_data=load_pdf(r"C:\Users\91939\Desktop\Medicalchatbot-with-llama2\data")

In [13]:
#exracted_data

[]

In [6]:
#create text chunks
def text_split(exracted_data):
    text_spilter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    text_chunks=text_spilter.split_documents(exracted_data)

    return text_chunks

In [7]:
text_chunks=text_split(exracted_data)
print(len(text_chunks))

5860


In [8]:
#download embedding model
def download_hugging_face_embeddings():
   embeddings = HuggingFaceEmbeddings(model_name="sentence-transformer/all-MiniLM-L6-v2")
   
   return embeddings


In [9]:
embeddings= download_hugging_face_embeddings()


ImportError: Could not import sentence_transformers python package. Please install it with `pip install sentence_transformers`.

In [ ]:
#!pip install --upgrade huggingface_hub sentence-transformers transformers


In [ ]:
query_res=embeddings.embed_query("Hello World")
print("lenght",len(query_res))

In [ ]:


# Create an instance of Pinecone with your API key
pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))

my_index="medicochatbot"

# Check if 'my_index' exists and create if not
if 'my_index' not in pc.list_indexes().names():
    pc.create_index(
        name='my_index',
        dimension=384,  # Use the correct dimension for your embeddings
        metric='cosine',  # You can change the metric to 'cosine', 'dot-product', etc.
        spec=ServerlessSpec(
            cloud='aws',  # Specify the cloud provider (e.g., 'aws', 'gcp', 'azure')
            region='us-east-1'  # Specify the region'
        )
    )




#Creating Embeddings for Each of The Text Chunks & storing
docsearch=Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, my_index=my_index)

In [ ]:
docsearch=Pinecone.from_existing_index(my_index, embeddings)

query = "What are Allergies"

docs=docsearch.similarity_search(query, k=3)

print("Result", docs)

In [ ]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [ ]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [ ]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [ ]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [ ]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])